In [ ]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from keras import regularizers
import numpy as np
import spacy
import os
import pandas as pd
import kagglehub

path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")
files_in_dataset = os.listdir(path)
dataset_file_path = os.path.join(path, "spam.csv")
df = np.array(pd.read_csv(dataset_file_path, encoding='latin-1'))


In [14]:
nlp = spacy.load("en_core_web_sm")
for i in range(df.shape[0]):
  doc = nlp(df[i,1])
  df[i,1] = " ".join([token.lemma_ for token in doc])

In [15]:
df[:,0][df[:,0] == "ham"] = 0
df[:,0][df[:,0] == "spam"] = 1

In [16]:
from sklearn.model_selection import train_test_split
x_train,  x_test, y_train, y_test = train_test_split(df[:,1], df[:,0], test_size = 0.2, random_state = 30)

In [17]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [18]:
model = Sequential([
    tf.keras.Input(shape=(6843,)),
    Dense(units = 50, activation = "relu",  kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(units = 10, activation = "relu",  kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(units = 1, activation = "sigmoid"),
  ]
)

In [ ]:
model.summary()

In [20]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
)

In [ ]:
x_train.sort_indices()
model.fit(
    x_train, y_train.astype(int), epochs = 25
)

In [ ]:
y_pred = model.predict(x_test)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.astype(int), y_pred.astype(int)))

In [ ]:
mail = input("enter ur email: ")
doc = nlp(mail)
mail = " ".join([token.lemma_ for token in doc])
mail = vectorizer.transform([mail,])
if model.predict(mail) > 0.5:
  print("spam")
else:
  print("not spam")